Hello Kagglers, in this Notebook I will try to Build a SBBTree and do faeature selection with RFE.

Hope to gain experience in writing this..

> If this notebook is useful to you, please DO UPVOTE 🗳

<h2>Competition Goal</h2>

With  the provided thousands of sixty-second sequences of sensor data recorded from several hundred participants who could have been in either of two possible activity states. 

determine what state a participant was in from the sensor data.

<h1>Importing Libraries</h1>

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode
init_notebook_mode(connected = True)
import warnings
warnings.filterwarnings('ignore')

<h2>Loading Data</h2>

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
test = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
submission = pd.read_csv("../input/tabular-playground-series-apr-2022/sample_submission.csv")
labels = pd.read_csv("../input/tabular-playground-series-apr-2022/train_labels.csv")

train

Data Description from kaggle 

train.csv 

*  sequence - a unique id for each sequence

* subject - a unique id for the subject in the experiment

* step - time step of the recording, in one second intervals

* sensor_00 - sensor_12 - the value for each of the thirteen sensors at that time step

* state - the value for each of the thirteen sensors at that time step ##


train_labels.csv - the class label for each sequence.

* sequence - the unique id for each sequence.

* state - the state associated to each sequence. This is the target which you are trying to predict.


test.csv - the test set. For each of the ~12,000 sequences, you should predict a value for that sequence's state.

In [ ]:
# give you an quike insght into the train data 
# including count,mean,std,min,25%,50%,75% and max value
train.describe()

In [ ]:
# run the code below to check if missing data exits
train.isnull().sum(axis=0)

<h3>adding labels to train data</h3>

In [ ]:
labels.head()

In [ ]:
train =train.merge(labels,how='left', on=["sequence"])
train.head()

<h1>Feature Seeking bewteen Target and Sensors Data</h1>


below I will test the 'mean', 'max', 'min', 'var', 'mad', 'sum', 'median' value of the data hoping to dig anything valuable

> the codes below are inspired by C4rl05/V with her work [https://www.kaggle.com/code/cv13j0/tps-apr-2022-xgboost-model](http://)

In [ ]:
def aggregated_features(df, aggregation_cols = ['sequence'], prefix = ''):
    agg_strategy = {'sensor_00': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_01': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_02': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_03': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_04': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_05': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_06': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_07': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_08': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_09': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_10': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_11': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                    'sensor_12': ['mean', 'max', 'min', 'var', 'mad', 'sum', 'median'],
                   }
    group = df.groupby(aggregation_cols).aggregate(agg_strategy)
    group.columns = ['_'.join(col).strip() for col in group.columns]
    group.columns = [str(prefix) + str(col) for col in group.columns]
    group.reset_index(inplace = True)
    
    temp = (df.groupby(aggregation_cols).size().reset_index(name = str(prefix) + 'size'))
    group = pd.merge(temp, group, how = 'left', on = aggregation_cols,)
    return group

In [ ]:
train_merge_data = aggregated_features(train, aggregation_cols = ['sequence', 'subject'])
test_merge_data = aggregated_features(test, aggregation_cols = ['sequence', 'subject'])

In [ ]:
train_subjects_merge_data = aggregated_features(train, aggregation_cols = ['subject'], prefix = 'subject_')
test_subjects_merge_data = aggregated_features(test, aggregation_cols = ['subject'], prefix = 'subject_')

up to now we have a clear view of the values of sensors 

In [ ]:
train_subjects_merge_data.head()

<h3>Merging the Datasets before Training</h3>

In [ ]:
train_merge_data = train_merge_data.merge(labels, how = 'left', on = 'sequence')

In [ ]:
train_merge_data = train_merge_data.merge(train_subjects_merge_data, how = 'left', on = 'subject')
test_merge_data = test_merge_data.merge(test_subjects_merge_data, how = 'left', on = 'subject')
train_merge_data.head()

In [ ]:
test_merge_data.head()

<h3>Post Processing the Information for the Model</h3>

In [ ]:
ignore = ['sequence', 'state', 'subject']
features = [feat for feat in train_merge_data.columns if feat not in ignore]
target_feature = 'state'

<h3>Train - Test Split </h3>

you may do cross-validation too.

In [ ]:
%%time
from sklearn.model_selection import train_test_split
test_size_pct = 0.3
X_train, X_valid, y_train, y_valid = train_test_split(
                                train_merge_data[features], 
                                train_merge_data[target_feature], 
                                test_size = test_size_pct, 
                                random_state = 2022)

<h1>Build SBBTree Model

In [ ]:
import lightgbm as lgb
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [ ]:
class SBBTree():
    """
        SBBTree
        Stacking,Bootstap,Bagging
    """
    def __init__(
                    self, 
                    params,
                    stacking_num,
                    bagging_num,
                    bagging_test_size,
                    num_boost_round,
                    early_stopping_rounds
                ):
        """
            Initializes the SBBTree.
            Args:
              params : lgb params.
              stacking_num : k_flod stacking.
              bagging_num : bootstrap num.
              bagging_test_size : bootstrap sample rate.
              num_boost_round : boost num.
              early_stopping_rounds : early_stopping_rounds.
        """
        self.params = params
        self.stacking_num = stacking_num
        self.bagging_num = bagging_num
        self.bagging_test_size = bagging_test_size
        self.num_boost_round = num_boost_round
        self.early_stopping_rounds = early_stopping_rounds

        self.model = lgb
        self.stacking_model = []
        self.bagging_model = []

    def fit(self, X, y):
        """ fit model. """
        if self.stacking_num > 1:
            layer_train = np.zeros((X.shape[0], 2))
            self.SK = KFold(n_splits=self.stacking_num, shuffle=True, random_state=16)
            for k,(train_index, test_index) in enumerate(self.SK.split(X, y)):
                X_train = X.iloc[train_index]
                y_train = y.iloc[train_index]
                X_test = X.iloc[test_index]
                y_test = y.iloc[test_index]

                lgb_train = lgb.Dataset(X_train, y_train)
                lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

                gbm = lgb.train(self.params,
                            lgb_train,
                            num_boost_round=self.num_boost_round,
                            valid_sets=lgb_eval,
                            early_stopping_rounds=self.early_stopping_rounds)

                self.stacking_model.append(gbm)

                pred_y = gbm.predict(X_test, num_iteration=gbm.best_iteration)
                layer_train[test_index, 1] = pred_y

            X = np.hstack((X, layer_train[:,1].reshape((-1,1)))) 
        else:
            pass
        for bn in range(self.bagging_num):
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=self.bagging_test_size, random_state=bn)

            lgb_train = lgb.Dataset(X_train, y_train)
            lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

            gbm = lgb.train(self.params,
                        lgb_train,
                        num_boost_round=6000,
                        valid_sets=lgb_eval,
                        early_stopping_rounds=200)

            self.bagging_model.append(gbm)

    def predict(self, X_pred):
        """ predict test data. """
        if self.stacking_num > 1:
            test_pred = np.zeros((X_pred.shape[0], self.stacking_num))
            for sn,gbm in enumerate(self.stacking_model):
                pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
                test_pred[:, sn] = pred
            X_pred = np.hstack((X_pred, test_pred.mean(axis=1).reshape((-1,1))))  
        else:
            pass 
        for bn,gbm in enumerate(self.bagging_model):
            pred = gbm.predict(X_pred, num_iteration=gbm.best_iteration)
            if bn == 0:
                pred_out=pred
            else:
                pred_out+=pred
        return pred_out/self.bagging_num

<h2>test the model with diff parameters here

In [ ]:
"""
    TEST CODE
"""
# from sklearn.datasets import make_classification
# from sklearn.datasets import load_breast_cancer
# from sklearn.datasets import make_gaussian_quantiles
# from sklearn import metrics
# from sklearn.metrics import f1_score
# # X, y = make_classification(n_samples=1000, n_features=25, n_clusters_per_class=1, n_informative=15, random_state=1)
# X, y = make_gaussian_quantiles(mean=None, cov=1.0, n_samples=1000, n_features=50, n_classes=2, shuffle=True, random_state=2)
# # data = load_breast_cancer()
# # X, y = data.data, data.target
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# params = {
#         'task': 'train',
#         'boosting_type': 'gbdt',
#         'objective': 'binary',
#         'metric': 'auc',
#         'num_leaves': 9,
#         'learning_rate': 0.03,
#         'feature_fraction_seed': 2,
#         'feature_fraction': 0.9,
#         'bagging_fraction': 0.8,
#         'bagging_freq': 5,
#         'min_data': 20,
#         'min_hessian': 1,
#         'verbose': 0,
#         'silent': 0
#         }
# # test 1
# model = SBBTree(params=params, stacking_num=2, bagging_num=1,  bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
# model.fit(X,y)
# X_pred = X[0].reshape((1,-1))
# pred=model.predict(X_pred)
# print('pred')
# print(pred)
# print('TEST 1 ok')


# # test 1
# model = SBBTree(params, stacking_num=1, bagging_num=1, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
# model.fit(X_train,y_train)
# pred1=model.predict(X_test)

# # test 2 
# model = SBBTree(params, stacking_num=1, bagging_num=3, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
# model.fit(X_train,y_train)
# pred2=model.predict(X_test)

# # test 3 
# model = SBBTree(params, stacking_num=5, bagging_num=1, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
# model.fit(X_train,y_train)
# pred3=model.predict(X_test)

# # test 4 
# model = SBBTree(params, stacking_num=5, bagging_num=3, bagging_test_size=0.33, num_boost_round=10000, early_stopping_rounds=200)
# model.fit(X_train,y_train)
# pred4=model.predict(X_test)

# fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred1, pos_label=2)
# print('auc: ',metrics.auc(fpr, tpr))

# fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred2, pos_label=2)
# print('auc: ',metrics.auc(fpr, tpr))

# fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred3, pos_label=2)
# print('auc: ',metrics.auc(fpr, tpr))

# fpr, tpr, thresholds = metrics.roc_curve(y_test+1, pred4, pos_label=2)
# print('auc: ',metrics.auc(fpr, tpr))

In [ ]:
params = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 2667,
        'learning_rate': 0.03,
        'feature_fraction_seed': 2,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'min_data': 20,
        'min_hessian': 1,
        'verbose': 0,
#         'silent': 0,
        'n_estimators': 8126,
        'max_depth': 6,
        'subsample': 0.96,
        'gamma': 1.40
        }

model = SBBTree(params=params,
                stacking_num=7,
                bagging_num=4,
                bagging_test_size=0.25,
                num_boost_round=8000,
                early_stopping_rounds=256)

In [ ]:
X_train.columns

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

rfe_selector = RFE(estimator=LogisticRegression(), n_features_to_select=135)
selector=rfe_selector.fit(X_train, y_train)
print(selector.support_)
print(selector.ranking_)
# rfe_support = rfe_selector.get_support()
# rfe_feature = X.loc[:,rfe_support].columns.tolist()
# print(str(len(rfe_feature)), 'selected features')


In [ ]:
rfe_support = selector.get_support()
rfe_feature = X_train.loc[:,rfe_support].columns.tolist()
print(str(len(rfe_feature)), 'selected features')
rfe_feature

In [ ]:
model.fit(X_train[rfe_feature], y_train)

In [ ]:
preds = model.predict(test_merge_data[rfe_feature])

In [ ]:
preds

In [ ]:
submission['state'] = preds
submission.to_csv('my_submission_ty.csv', index = False)